In [42]:
from pulp import *
import pandas as pd

In [43]:
# Cargar datos
df_vehiculos = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')
df_clientes = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_distancias = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')

df_distancias.index = df_distancias.columns


In [44]:
# Preparación datos para PuLP
capacidades = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["capacidad_kg"]))
costos = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["costo_km"]))
autonomias = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["autonomia_km"]))

pedidos = dict(zip(df_clientes["cliente"], df_clientes["order_demand"]))
pedidos["Almacén"] = 0  # Aseguramos que el almacén no tenga demanda

distancias = df_distancias.stack().to_dict()  # Convierte a un diccionario (i, j): distancia

clientes = df_clientes["cliente"].tolist() + ["Almacén"]  # Incluir el almacén como un cliente especial
vehiculos = df_vehiculos["vehiculo_id"].tolist()

# Crear el problema
problema = LpProblem("Ruteo_de_Vehiculos", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("x", [(i, j, k) for i in clientes for j in clientes for k in vehiculos], cat="Binary")
q = LpVariable.dicts("q", [(i, k) for i in clientes for k in vehiculos], lowBound=0)

# Función objetivo: minimizar costo total
problema += lpSum(
    x[i, j, k] * costos[k] * distancias.get((i, j), 0)
    for i in clientes for j in clientes for k in vehiculos
)

# Restricciones
# Cada cliente debe ser atendido exactamente una vez
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(x[i, j, k] for j in clientes for k in vehiculos) == 1

# Respetar la capacidad de los vehículos
for k in vehiculos:
    problema += lpSum(q[i, k] for i in clientes) <= capacidades[k]

# Satisfacer la demanda de los clientes
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(q[i, k] for k in vehiculos) == pedidos[i]

# Autonomía de los vehículos, considerando el viaje de ida y vuelta al almacén
for k in vehiculos:
    problema += lpSum(x[i, j, k] * distancias.get((i, j), 0) for i in clientes for j in clientes) \
                + lpSum(x[i, 'Almacén', k] * distancias.get((i, 'Almacén'), 0) for i in clientes if i != 'Almacén') \
                + lpSum(x['Almacén', j, k] * distancias.get(('Almacén', j), 0) for j in clientes if j != 'Almacén') \
                <= autonomias[k]

# Flujo de vehículos: entrada = salida, incluido el almacén
for i in clientes:
    for k in vehiculos:
        if i != 'Almacén':
            problema += lpSum(x[i, j, k] for j in clientes) == lpSum(x[j, i, k] for j in clientes)
        else:
            # Para el almacén, la suma de las salidas debe ser igual a la suma de las entradas
            problema += lpSum(x[i, j, k] for j in clientes if j != i) == lpSum(x[j, i, k] for j in clientes if j != i)

# Restricción de Inicio desde el Almacén
for k in vehiculos:
    problema += lpSum(x['Almacén', j, k] for j in clientes if j != 'Almacén') == 1

# Restricción de Retorno al Almacén
for k in vehiculos:
    problema += lpSum(x[i, 'Almacén', k] for i in clientes if i != 'Almacén') == 1

# Asegurar que no haya bucles directos entre clientes sin pasar por el almacén
for i in clientes:
    if i != 'Almacén':
        for j in clientes:
            if j != 'Almacén' and i != j:
                for k in vehiculos:
                    problema += x[i, j, k] + x[j, i, k] <= 1  # No puede haber un viaje de ida y vuelta entre dos clientes

# Restricción adicional para evitar bucles
for k in vehiculos:
    for i in clientes:
        if i != 'Almacén':
            problema += lpSum(x[j, i, k] for j in clientes) - lpSum(x[i, j, k] for j in clientes if j != i) == 0

# Asegurar que q[i, k] solo tenga valor si el vehículo k va al cliente i
for i in clientes:
    for k in vehiculos:
        problema += q[i, k] <= lpSum(x[i, j, k] for j in clientes) * pedidos[i]

# Resolver el problema con un tiempo límite
solver = PULP_CBC_CMD(msg=1, timeLimit=60)  # 300 segundos de límite de tiempo
problema.solve(solver)
print(LpStatus[problema.status])

# Debug: Imprimir decisiones de rutas y cargas
for i in clientes:
    for j in clientes:
        for k in vehiculos:
            if x[i, j, k].value() == 1:
                print(f"Vehículo {k} va de {i} a {j}")


# Verificación de Demanda vs Capacidad Total
total_demand = sum(pedidos[i] for i in clientes if i != 'Almacén')
total_capacity = sum(capacidades[k] for k in vehiculos)
print(f"Total demanda: {total_demand}, Total capacidad: {total_capacity}")

In [45]:
# Crear el problema
problema = LpProblem("Ruteo_de_Vehiculos", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("x", [(i, j, k) for i in clientes for j in clientes for k in vehiculos], cat="Binary")
q = LpVariable.dicts("q", [(i, k) for i in clientes for k in vehiculos], lowBound=0)

# Función objetivo: minimizar costo total
problema += lpSum(
    x[i, j, k] * costos[k] * distancias.get((i, j), 0)
    for i in clientes for j in clientes for k in vehiculos
)

# Restricciones
# Cada cliente debe ser atendido exactamente una vez
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(x[i, j, k] for j in clientes for k in vehiculos) == 1

# Respetar la capacidad de los vehículos
for k in vehiculos:
    problema += lpSum(q[i, k] for i in clientes) <= capacidades[k]

# Satisfacer la demanda de los clientes
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(q[i, k] for k in vehiculos) == pedidos[i]

# Autonomía de los vehículos, considerando el viaje de ida y vuelta al almacén
for k in vehiculos:
    problema += lpSum(x[i, j, k] * distancias.get((i, j), 0) for i in clientes for j in clientes) <= autonomias[k]

# Flujo de vehículos: entrada = salida, incluido el almacén
for i in clientes:
    for k in vehiculos:
        problema += lpSum(x[i, j, k] for j in clientes if j != i) == lpSum(x[j, i, k] for j in clientes if j != i)

# Restricción de inicio desde el almacén
for k in vehiculos:
    problema += lpSum(x['Almacén', j, k] for j in clientes if j != 'Almacén') == 1

# Restricción de retorno al almacén
for k in vehiculos:
    problema += lpSum(x[i, 'Almacén', k] for i in clientes if i != 'Almacén') == 1

# Evitar bucles entre clientes sin pasar por el almacén
for i in clientes:
    if i != 'Almacén':
        for j in clientes:
            if j != 'Almacén' and i != j:
                for k in vehiculos:
                    problema += x[i, j, k] + x[j, i, k] <= 1

# Asegurar que q[i, k] solo tenga valor si el vehículo k va al cliente i
for i in clientes:
    for k in vehiculos:
        problema += q[i, k] <= pedidos[i] * lpSum(x[i, j, k] for j in clientes)

# Resolver el problema con un tiempo límite
solver = PULP_CBC_CMD(msg=1, timeLimit=300)  # 300 segundos de límite de tiempo
problema.solve(solver)

# Mostrar el estado del problema
print("Estado del problema:", LpStatus[problema.status])

# Verificar si todos los clientes fueron visitados
clientes_visitados = set()
for i in clientes:
    for j in clientes:
        for k in vehiculos:
            if x[i, j, k].value() == 1:
                print(f"{k}: {i} -> {j}")
                clientes_visitados.add(i)
                clientes_visitados.add(j)

clientes_no_visitados = set(clientes) - clientes_visitados - {"Almacén"}
if clientes_no_visitados:
    print("Los siguientes clientes no fueron visitados:", clientes_no_visitados)
else:
    print("Todos los clientes fueron visitados.")

Estado del problema: Optimal
2: Cliente_1 -> Almacén
3: Cliente_2 -> Cliente_2
5: Cliente_3 -> Cliente_3
4: Cliente_4 -> Almacén
2: Cliente_5 -> Cliente_5
1: Cliente_6 -> Almacén
6: Cliente_7 -> Cliente_7
6: Cliente_8 -> Cliente_8
5: Cliente_9 -> Almacén
3: Cliente_10 -> Cliente_10
3: Cliente_11 -> Almacén
3: Cliente_12 -> Cliente_12
4: Cliente_13 -> Cliente_13
6: Cliente_14 -> Almacén
3: Cliente_15 -> Cliente_15
5: Cliente_16 -> Cliente_16
4: Cliente_17 -> Cliente_17
2: Cliente_18 -> Cliente_18
2: Cliente_19 -> Cliente_19
1: Cliente_20 -> Cliente_20
2: Almacén -> Cliente_1
4: Almacén -> Cliente_4
1: Almacén -> Cliente_6
5: Almacén -> Cliente_9
3: Almacén -> Cliente_11
6: Almacén -> Cliente_14
Todos los clientes fueron visitados.


In [46]:
# Resultados detallados
for k in vehiculos:
    print(f"\nVehículo {k}:")
    ruta = []
    carga_total = 0
    distancia_total = 0
    nodo_actual = 'Almacén'

    while True:
        ruta.append(nodo_actual)
        siguiente_nodo = next((j for j in clientes if x[nodo_actual, j, k].value() == 1), None)
        if siguiente_nodo is None or siguiente_nodo == 'Almacén':
            break
        carga_total += pedidos[siguiente_nodo]
        distancia_total += distancias[nodo_actual, siguiente_nodo]
        nodo_actual = siguiente_nodo

    ruta.append('Almacén')
    distancia_total += distancias.get((nodo_actual, 'Almacén'), 0)

    if len(ruta) > 2:
        print("  Ruta:", " -> ".join(ruta))
        print(f"  Carga total: {carga_total} kg")
        print(f"  Distancia total: {distancia_total:.2f} km")
        print(f"  Capacidad: {capacidades[k]} kg, Autonomía: {autonomias[k]} km")
    else:
        print("  No se asignó ruta a este vehículo.")

# Resumen de la capacidad vs demanda
total_demand = sum(pedidos[i] for i in clientes if i != 'Almacén')
total_capacity = sum(capacidades[k] for k in vehiculos)
print(f"\nDemanda total: {total_demand} kg")
print(f"Capacidad total disponible: {total_capacity} kg")


Vehículo 1:
  Ruta: Almacén -> Cliente_6 -> Almacén
  Carga total: 908 kg
  Distancia total: 5.39 km
  Capacidad: 2026 kg, Autonomía: 603 km

Vehículo 2:
  Ruta: Almacén -> Cliente_1 -> Almacén
  Carga total: 909 kg
  Distancia total: 7.22 km
  Capacidad: 4362 kg, Autonomía: 630 km

Vehículo 3:
  Ruta: Almacén -> Cliente_11 -> Almacén
  Carga total: 942 kg
  Distancia total: 5.82 km
  Capacidad: 4881 kg, Autonomía: 664 km

Vehículo 4:
  Ruta: Almacén -> Cliente_4 -> Almacén
  Carga total: 980 kg
  Distancia total: 6.73 km
  Capacidad: 3321 kg, Autonomía: 514 km

Vehículo 5:
  Ruta: Almacén -> Cliente_9 -> Almacén
  Carga total: 886 kg
  Distancia total: 2.10 km
  Capacidad: 10000 kg, Autonomía: 350 km

Vehículo 6:
  Ruta: Almacén -> Cliente_14 -> Almacén
  Carga total: 932 kg
  Distancia total: 8.06 km
  Capacidad: 3129 kg, Autonomía: 791 km

Demanda total: 18738 kg
Capacidad total disponible: 27719 kg


# Resultados
for k in vehiculos:
    print(f"Vehículo {k}:")
    ruta = []
    visited = set()
    start = 'Almacén'
    while True:
        if start in visited and start != 'Almacén':
            break  # Evitar bucles infinitos
        visited.add(start)
        next_node = next((j for j in clientes if j != start and x[start, j, k].value() == 1), None)
        if next_node is None:
            break  # Si no hay más movimientos, el vehículo ha vuelto al almacén
        ruta.append(start)
        start = next_node
        if start == 'Almacén':
            ruta.append(start)
            break
    if ruta:
        print(f"  Ruta: {' -> '.join(ruta)}")
    else:
        print("  Ruta: No se ha asignado una ruta a este vehículo")
    carga_total = sum(pedidos[i] for i in ruta if i != 'Almacén')
    print(f"  Carga total: {carga_total} kg")
    total_distance = sum(x[i, j, k].value() * distancias.get((i, j), 0) for i in clientes for j in clientes if x[i, j, k].value() == 1)
    print(f"  Distancia total: {total_distance} km")
    print(f"  Autonomía: {autonomias[k]} km")
    print(f"  Capacidad: {capacidades[k]} kg")
    print()